In [1]:
#2018-08-07 13:13:04.742507
%load_ext metapack.jupyter.magic

In [2]:
CACHE_DIR='/Users/eric/Library/Application Support/metapack/'
RESOURCE_NAME='stations'
RESOLVED_URL='file:///Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-beachwatch/notebooks/Add_Features.ipynb#stations'
WORKING_DIR='/Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-beachwatch'
METATAB_DOC='metapack+file:///Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-beachwatch/metadata.csv'
METATAB_WORKING_DIR='/Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-beachwatch'
METATAB_PACKAGE='metapack+file:///Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-beachwatch/'
name='stations'
description='Measurement stations'
url='notebooks/Add_Features.ipynb#stations'

In [3]:
METAPACK_BUILDING=True

In [4]:
import matplotlib.pyplot as plt 
import metapack as mp
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline

from metapack.appurl import SearchUrl 
SearchUrl.initialize()

In [5]:
pkg = mp.jupyter.open_package()
pkg

MetatabFileNotFound: Failed to get Metatabfile for reference: 'metapack+file:///Users/eric/proj/virt-proj/water/water-datasets/derived/sandiegodata.org-beachwatch/_packages/sandiegodata.org-beachwatch-1/metadata.csv' 

In [3]:
beachwatch_source = pkg.reference('beachwatch_source').read_csv()
# It looks like the prefix of the station code groups stations, maybe into watersheds. 


In [4]:
measurement_cols = [
 'sampledate',
 'collectiontime',
 'stationcode',
 'stationgroup',
 'result',
 'methodname',
 'analyte',
 'unit',
 'labbatch',                
 'resultqualcode',
 'qacode',
 'sampleagency',
 'labagency',
 'submittingagency'
]

station_cols = [
 'stationname',
 'stationcode',
 'stationgroup',
 'targetlatitude',
 'targetlongitude',
]

beachwatch_source['stationgroup'] = beachwatch_source.stationcode.str[:2]

obs = beachwatch_source[measurement_cols]
stations = beachwatch_source[station_cols].drop_duplicates()


In [5]:
len(obs), len(stations)

(202257, 172)

# Group Statistics and Measure Codes

In [6]:
# Create a dataset of all of the unique measure codes, then give them unique values
_1 = obs[['analyte','methodname','unit']].drop_duplicates()
_1['measure_code'] = 0
_1 = _1.set_index(list(_1.columns)[:-1]).sort_index()
_1['measure_code'] = _1.reset_index().index # Reset_index() renumbers the index
measure_codes = _1
measure_codes.head()

measure_code
analyte         methodname unit                    
Coliform, Fecal MTF        MPN/100 mL             0
                           cfu/100mL              1
                SM 9221 E  MPN/100 mL             2
                           cfu/100mL              3
                SM 9222 B  MPN/100 mL             4

In [7]:
obs_1 = obs.reset_index().set_index(['analyte','methodname','unit']).join(measure_codes).reset_index()
obs_1.head()

,analyte,methodname,unit,index,sampledate,collectiontime,stationcode,stationgroup,result,labbatch,resultqualcode,qacode,sampleagency,labagency,submittingagency,measure_code
0,"Coliform, Fecal",MTF,MPN/100 mL,4,1999-05-26,2018-08-07,EH-010,EH,20.0,SH-5/26/1999,<,NR,SDCDEH,SDCDEH,SDCDEH,0
1,"Coliform, Fecal",MTF,MPN/100 mL,11,1999-10-13,2018-08-07,EH-010,EH,0.0,SH-10/13/1999,=,NR,SDCDEH,SDCDEH,SDCDEH,0
2,"Coliform, Fecal",MTF,MPN/100 mL,14,1999-10-26,2018-08-07,EH-010,EH,20.0,SH-10/26/1999,<,NR,SDCDEH,SDCDEH,SDCDEH,0
3,"Coliform, Fecal",MTF,MPN/100 mL,19,2000-03-21,2018-08-07,EH-010,EH,20.0,SH-3/21/2000,<,NR,SDCDEH,SDCDEH,SDCDEH,0
4,"Coliform, Fecal",MTF,MPN/100 mL,25,2000-05-24,2018-08-07,EH-010,EH,230.0,SH-5/24/2000,=,NR,SDCDEH,SDCDEH,SDCDEH,0


In [8]:
# Compute stats for each group of
obs_1['lresult'] = obs_1.result.apply(lambda x: np.nan if x == 0 else np.log(x))

_g = obs_1[['stationcode','measure_code','sampledate', 'result', 'lresult']].groupby(['stationcode','measure_code'])
_1 = _g.agg({'result':['count','std','mean'], 
             'lresult':['std','mean']})
    
# _1 now has multi-level column names, so we need to compress them. 
_1.columns = [ '_'.join(e).replace('result_','result_group_') for e in _1.columns.get_values()]

# compute quantiles 
_2 = obs_1[['stationcode','measure_code', 'result', 'lresult']]\
        .groupby(['stationcode','measure_code'])\
        .quantile([.25,.5, .75])
        
_2.index.names = ['stationcode', 'measure_code', 'quantile']
_2 = _2.unstack(-1)

_2.columns = [ '_'.join( str(ee) for ee in e).replace('0.','') for e in _2.columns.get_values()]

_2.rename(columns={
    'lresult_25': 'lresult_group_25pctl',
    'lresult_5':  'lresult_group_median',
    'lresult_75': 'lresult_group_75pctl',
    'result_25':  'result_group_25pctl',
    'result_5':   'result_group_median',
    'result_75':  'result_group_75pctl',
}, inplace=True)

# Finally, join
obs_m = obs_1.reset_index().set_index(['stationcode','measure_code']).join(_1).join(_2).reset_index().copy()
obs_m.head()


,stationcode,measure_code,level_0,analyte,methodname,unit,index,sampledate,collectiontime,stationgroup,...,result_group_std,result_group_mean,lresult_group_std,lresult_group_mean,lresult_group_25pctl,lresult_group_median,lresult_group_75pctl,result_group_25pctl,result_group_median,result_group_75pctl
0,EH-010,0,0,"Coliform, Fecal",MTF,MPN/100 mL,4,1999-05-26,2018-08-07,EH,...,1647.469843,376.478873,1.439884,3.633361,2.995732,2.995732,2.995732,20.0,20.0,20.0
1,EH-010,0,1,"Coliform, Fecal",MTF,MPN/100 mL,11,1999-10-13,2018-08-07,EH,...,1647.469843,376.478873,1.439884,3.633361,2.995732,2.995732,2.995732,20.0,20.0,20.0
2,EH-010,0,2,"Coliform, Fecal",MTF,MPN/100 mL,14,1999-10-26,2018-08-07,EH,...,1647.469843,376.478873,1.439884,3.633361,2.995732,2.995732,2.995732,20.0,20.0,20.0
3,EH-010,0,3,"Coliform, Fecal",MTF,MPN/100 mL,19,2000-03-21,2018-08-07,EH,...,1647.469843,376.478873,1.439884,3.633361,2.995732,2.995732,2.995732,20.0,20.0,20.0
4,EH-010,0,4,"Coliform, Fecal",MTF,MPN/100 mL,25,2000-05-24,2018-08-07,EH,...,1647.469843,376.478873,1.439884,3.633361,2.995732,2.995732,2.995732,20.0,20.0,20.0


In [9]:
#list(obs_m.columns)

# Adding Features

In [10]:
obs_m['result_gt_median'] = (obs_m.result > obs_m.result_group_median).astype(int)
obs_m['result_gt_mean']   = (obs_m.result > obs_m.result_group_mean).astype(int)
obs_m['result_lte_25pctl']   = (obs_m.result <= obs_m.result_group_25pctl).astype(int)
obs_m['result_gte_75pctl']   = (obs_m.result >= obs_m.result_group_75pctl).astype(int)

obs_m['lresult_gt_lmedian'] = (obs_m.lresult > obs_m.lresult_group_median).astype(int)
obs_m['lresult_gt_lmean']   = (obs_m.lresult > obs_m.lresult_group_mean).astype(int)
obs_m['lresult_lte_25pctl']   = (obs_m.lresult <= obs_m.lresult_group_25pctl).astype(int)
obs_m['lresult_gte_75pctl']   = (obs_m.lresult >= obs_m.lresult_group_75pctl).astype(int)

In [11]:
obs_m.head().T

,0,1,2,3,4
stationcode,EH-010,EH-010,EH-010,EH-010,EH-010
measure_code,0,0,0,0,0
level_0,0,1,2,3,4
analyte,"Coliform, Fecal","Coliform, Fecal","Coliform, Fecal","Coliform, Fecal","Coliform, Fecal"
methodname,MTF,MTF,MTF,MTF,MTF
unit,MPN/100 mL,MPN/100 mL,MPN/100 mL,MPN/100 mL,MPN/100 mL
index,4,11,14,19,25
sampledate,1999-05-26 00:00:00,1999-10-13 00:00:00,1999-10-26 00:00:00,2000-03-21 00:00:00,2000-05-24 00:00:00
collectiontime,2018-08-07 00:00:00,2018-08-07 00:00:00,2018-08-07 00:00:00,2018-08-07 00:00:00,2018-08-07 00:00:00
stationgroup,EH,EH,EH,EH,EH


In [12]:
 list(obs_m.columns)
  
obs_m_cols =[
    'stationcode',
    'stationgroup',
    'sampledate',
    'collectiontime',
    'measure_code',
    'analyte',
    'methodname',
    'unit',
    'result',
    'result_group_count',
    'result_group_std',
    'result_group_mean',
    'result_group_25pctl',
    'result_group_median',
    'result_group_75pctl',
    'result_gt_median',
    'result_gt_mean',
    'result_lte_25pctl',
    'result_gte_75pctl',
    'lresult',
    'lresult_group_std',
    'lresult_group_mean',
    'lresult_group_25pctl',
    'lresult_group_median',
    'lresult_group_75pctl',
    'lresult_gt_lmedian',
    'lresult_gt_lmean',
    'lresult_lte_25pctl',
    'lresult_gte_75pctl',
    'labbatch',
    'resultqualcode',
    'qacode',
    'sampleagency',
    'labagency',
    'submittingagency'
]

obs_m = obs_m[obs_m_cols]
obs_m.head().T 

,0,1,2,3,4
stationcode,EH-010,EH-010,EH-010,EH-010,EH-010
stationgroup,EH,EH,EH,EH,EH
sampledate,1999-05-26 00:00:00,1999-10-13 00:00:00,1999-10-26 00:00:00,2000-03-21 00:00:00,2000-05-24 00:00:00
collectiontime,2018-08-07 00:00:00,2018-08-07 00:00:00,2018-08-07 00:00:00,2018-08-07 00:00:00,2018-08-07 00:00:00
measure_code,0,0,0,0,0
analyte,"Coliform, Fecal","Coliform, Fecal","Coliform, Fecal","Coliform, Fecal","Coliform, Fecal"
methodname,MTF,MTF,MTF,MTF,MTF
unit,MPN/100 mL,MPN/100 mL,MPN/100 mL,MPN/100 mL,MPN/100 mL
result,20,0,20,20,230
result_group_count,213,213,213,213,213


In [ ]:
%mt_materialize stations '/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-beachwatch-1' 

In [ ]:

%mt_materialize_all '/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-beachwatch-1'


In [ ]:

%mt_show_metatab



In [ ]:

%mt_show_libdirs

